## 作業
實作各種不一樣的方式來調整飽和 / 對比 / 明亮

1. 改變 color space 來調整飽和度
2. 實作直方圖均衡
3. alpha/ beta 調整對比 / 明亮

In [2]:
import cv2
import numpy as np

# Using lena.png
img = 'img/lena.png'

# default read as cv2.IMREAD_COLOR(彩色圖)
img_bgr = cv2.imread(img, cv2.IMREAD_COLOR)

## 調整飽和度

In [53]:
# for changing saturation, need to transfer color space to HSL (openCV : cv2.COLOR_BGR2HLS)
img_hls = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HLS)
change_percent = 0.2

# reducing 20% of saturation (saturation channel -> img_hls_down[:,:,-1])
img_hls_down = img_hls.astype('float32') # transfer to 'float32' type before changing saturation
img_hls_down[:,:,-1] = img_hls_down[:,:,-1] / 255 - change_percent # saturation/255(limit to 0~1),減少20%後可能有負數
img_hls_down[:,:,-1][img_hls_down[:,:,-1] < 0] = 0
img_hls_down[:,:,-1] = img_hls_down[:,:,-1]*255   # return to 0~255
img_hls_down = img_hls_down.astype('uint8')       # return to uint8 type

# increasing 20% of saturation
img_hls_up = img_hls.astype('float32') 
img_hls_up[:,:,-1] = img_hls_up[:,:,-1] / 255 + change_percent # saturation/255(limit to 0~1),增加20%後可能>1
img_hls_up[:,:,-1][img_hls_up[:,:,-1] > 1] = 1
img_hls_up[:,:,-1] = img_hls_up[:,:,-1]*255 
img_hls_up = img_hls_up.astype('uint8') 

# transfer color space return to BGR
img_hls_down = cv2.cvtColor(img_hls_down, cv2.COLOR_HLS2BGR)
img_hls_up = cv2.cvtColor(img_hls_up, cv2.COLOR_HLS2BGR)

# 橫向組合圖片
img_hls_new = np.hstack((img_bgr, img_hls_down, img_hls_up))

while True :
    # display of img_hsv, named as 'changing saturation'
    cv2.imshow('changing saturation', img_hls_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

## 直方圖均衡(Histogram Equalization)
- case 1 : histogram equalization by GARY image
- case 2 : histogram equalization by value channel of HSV
- case 3 : histogram equalization by each channel of BGR

In [10]:
# case 1 : histogram equalization by GARY image
# transfer color space to GRAY image -> 2D array
img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

# histogram equalization by GARY image -> 增強對比
img_gray_equal = cv2.equalizeHist(img_gray)

# 橫向組合圖片
img_gray_equal_new = np.hstack((img_gray, img_gray_equal))

while True :
    # display of img_gray_equal_new, named as 'gray equal histogram'
    cv2.imshow('gray equal histogram', img_gray_equal_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

In [63]:
# case 2 : histogram equalization by value channel of HSV
# transfer color space to HSV image (3D array)
img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
img_hsv_v_equal = img_hsv.copy()

# histogram equalization by value(brightness) channel of HSV image (2D array) -> 增強對比
img_hsv_v_equal[:,:,2] = cv2.equalizeHist(img_hsv[:,:,2])

# transfer color space return to BGR
img_hsv_v_equal = cv2.cvtColor(img_hsv_v_equal, cv2.COLOR_HSV2BGR)

# 橫向組合圖片
img_hsv_v_equal_new = np.hstack((img_bgr, img_hsv_v_equal))

while True :
    # display of img_hsv_v_equal_new, named as 'hsv_v equal histogram'
    cv2.imshow('hsv_v equal histogram', img_hsv_v_equal_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

In [3]:
# case 3 : histogram equalization by each channel of BGR

# histogram equalization by each bgr channel(2D array)(512,512) -> 增強對比
img_bgr_channel = [img_bgr[:,:,0], img_bgr[:,:,1], img_bgr[:,:,2]]
equalHist_by_bgr = [cv2.equalizeHist(i) for i in img_bgr_channel]
# stack of img_bgr[:,:,0], img_bgr[:,:,1], img_bgr[:,:,2]) -> result become as 3D array
# axis=-1 means the new axis will be the last dimension(512,512,3)
img_bgr_equal = np.stack((equalHist_by_bgr), axis=-1)

# 橫向組合圖片
img_bgr_equal_new = np.hstack((img_bgr, img_bgr_equal))

while True :
    # display of img_bgr_equal_new, named as 'bgr equal histogram'
    cv2.imshow('bgr equal histogram', img_bgr_equal_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

## 調整對比度 & 明亮度

In [57]:
# adjusting contrast by alpha(1~3), adjusting lightness by beta(0~255)
add_contrast = cv2.convertScaleAbs(img_bgr, alpha=1.0, beta=50.0)
add_lightness = cv2.convertScaleAbs(img_bgr, alpha=2.0, beta=0.0)

# 橫向組合圖片
img_contrast_lightness_new = np.hstack((img_bgr, add_contrast, add_lightness))

while True :
    # display of img_gray_equal_new, named as 'gray equal histogram'
    cv2.imshow('adjusting contrast & lightness', img_contrast_lightness_new)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break